Real data input

In [ ]:
import pandas as pd
import datetime

def process_csv(filepath, product_index):
    trade_data = pd.read_csv(filepath)

    trade_data["TIME_PERIOD"] = pd.to_datetime(trade_data["TIME_PERIOD"], format = '%Y-%m').dt.date
    trade_data = trade_data.rename(columns={"partner": "PARTNER_codes", "declarant": "DECLARANT_codes"})
    trade_data = trade_data.drop_duplicates()
    #trade_data = trade_data.dropna()

    products = trade_data["product"].drop_duplicates().to_list()
    product_filter = products[product_index]

    partner_countries = pd.read_csv("trade_data/partners.csv", sep = ";")

    trade_data = pd.merge(trade_data, partner_countries, on = ['PARTNER_codes'], how = 'inner')

    trade_data_pivot = trade_data.pivot(index=['PARTNER_Labels', 'TIME_PERIOD', 'product'],
                                        columns='indicators',
                                        values=['OBS_VALUE'])

    trade_data_pivot.columns = trade_data_pivot.columns.map('_'.join)
    trade_data_pivot = trade_data_pivot.reset_index()
    trade_data_pivot = trade_data_pivot.dropna()

    df = trade_data_pivot
    df_product_filtered = df[df['product'] == product_filter]

    return df_product_filtered

#Declare DataFrames for all good/productcode combinations
df_sunflower_oil_0 = process_csv("trade_data/sunflower_oil.csv", 0)
df_sunflower_oil_1 = process_csv("trade_data/sunflower_oil.csv", 1)
df_maize_corn_0 = process_csv("trade_data/maize_corn.csv", 0)
df_wheat_0 = process_csv("trade_data/wheat.csv", 0)
df_wheat_1 = process_csv("trade_data/wheat.csv", 1)
df_wheat_2 = process_csv("trade_data/wheat.csv", 2)

Univariate series (test) data input with n-steps

In [1]:
# Import packages
from numpy import array

In [11]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X = list()
    for i in range(len(sequence)):
        #find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x = sequence[i:end_ix]
        X.append(seq_x)
    return array(X)
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps = 3
# split into samples
X = split_sequence(raw_seq, n_steps)
# summarize the data
print(X)`

[[10 20 30]
 [20 30 40]
 [30 40 50]
 [40 50 60]
 [50 60 70]
 [60 70 80]]


In [41]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X = list()
    for i in range(np.shape(sequences)[1]):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > np.shape(sequences)[1]:
            break
        # gather input and output parts of the pattern
        bag_x= sequences[:,i:end_ix]
        X.append(bag_x)
    return array(X)

import numpy as np
# define input sequence
A = np.random.randint(1,10,size = (3,10))
# choose a number of time steps
n_steps = 3
print(A)
# split into samples
X = split_sequences(A, n_steps)
# summarize the data
print(X)

[[2 6 4 4 8 5 1 1 2 9]
 [7 7 9 3 9 7 8 6 8 9]
 [1 7 1 8 8 5 7 1 9 4]]
[[[2 6 4]
  [7 7 9]
  [1 7 1]]

 [[6 4 4]
  [7 9 3]
  [7 1 8]]

 [[4 4 8]
  [9 3 9]
  [1 8 8]]

 [[4 8 5]
  [3 9 7]
  [8 8 5]]

 [[8 5 1]
  [9 7 8]
  [8 5 7]]

 [[5 1 1]
  [7 8 6]
  [5 7 1]]

 [[1 1 2]
  [8 6 8]
  [7 1 9]]

 [[1 2 9]
  [6 8 9]
  [1 9 4]]]


In [48]:
list(range(0,np.shape(X)[0]))

[0, 1, 2, 3, 4, 5, 6, 7]

In [49]:
# split a dataframe into windowed dataframe
def WindowGenerator(df, n_steps):
    X = list()
    for i in range(df.shape[0]):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > df.shape[0]:
            break
        # gather input and output parts of the pattern
        window_x= df.iloc[i:end_ix]
        X.append(window_x)
    return array(X)

In [35]:
import pandas as pd
import numpy as np

# intialise data of lists.
data = {'Name':['a', 'b', 'c', 'd', 'e', 'f'],
        'Age':[20, 21, 19, 18,17,16],
        'Length':[203, 212, 196, 183,123,135]}

# Create DataFrame
df = pd.DataFrame(data)
print(df)

  Name  Age  Length
0    a   20     203
1    b   21     212
2    c   19     196
3    d   18     183
4    e   17     123
5    f   16     135


In [52]:
# split into samples
n_steps = 3
W = WindowGenerator(df, n_steps)
# summarize the data
print(W)
W.shape

[[['a' 20 203]
  ['b' 21 212]
  ['c' 19 196]]

 [['b' 21 212]
  ['c' 19 196]
  ['d' 18 183]]

 [['c' 19 196]
  ['d' 18 183]
  ['e' 17 123]]

 [['d' 18 183]
  ['e' 17 123]
  ['f' 16 135]]]


(4, 3, 3)